In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle

In [2]:
# Load all the trained model, scaler file, one hot encoder

model = load_model('model.h5')

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

with open('OneHot_Encoder_geo.pkl', 'rb') as file:
    label_encoder_geo = pickle.load(file)

with open('Label_Encoder_Gender.pkl', 'rb') as file:
    label_encoder_gender = pickle.load(file)

In [3]:
# Example input data
input_data = {
    'CreditScore': 600,
    'Geography': 'France',
    'Gender': 'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,
    'EstimatedSalary': 50000
}

In [4]:
geo_encoded = label_encoder_geo.transform([[input_data['Geography']]]).toarray()
print(geo_encoded)
geo_encoded_df = pd.DataFrame(geo_encoded, columns=label_encoder_geo.get_feature_names_out(['Geography'])) 
print(geo_encoded_df)

[[1. 0. 0.]]
   Geography_France  Geography_Germany  Geography_Spain
0               1.0                0.0              0.0


d:\Projects\ANN-Classification\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


In [15]:
label_encoder_geo.categories_[0] 

array(['France', 'Germany', 'Spain'], dtype=object)

In [5]:
input_data = pd.DataFrame([input_data])
input_data

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,Male,40,3,60000,2,1,1,50000


In [6]:
input_data['Gender'] = label_encoder_gender.transform([input_data['Gender']])
input_data

d:\Projects\ANN-Classification\venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,600,France,1,40,3,60000,2,1,1,50000


In [7]:
input_data = pd.concat([input_data.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_data

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,600,1,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [8]:
#  Scaling the input data

input_scaled=scaler.transform(input_data)
input_scaled

array([[-0.47252305,  0.91407657,  0.0906231 , -0.70903082, -0.27592043,
         0.79187296,  0.63599873,  0.97160303, -0.84394851,  0.98728088,
        -0.58166121, -0.56318378]])

In [9]:
# Predict churn 
prediction = model.predict(input_scaled)
print(prediction)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
[[0.03132934]]


In [12]:
prediction_proba = prediction[0][0]

In [13]:
print(prediction_proba)

0.03132934


In [14]:
if(prediction_proba>0.5):
    print("Customer will Churn")
else:
    print("Customer will not churn")

Customer will not churn
